# Création DataFrame IIT_CDIP_COLL_XML_FEATURES

## README
Après comparaison des différentes dataframes, il apparait que la base de données ayant la meilleure complétude et les odnnées naturellement les plus propres est celle contenue dans df_iit_cdip_coll_xml_ltdlwocr_features.

En choisissant de ne traiter un document que sur les seules informations qu'il contient (et donc sans les métadonnées telles que son emetteur, son récipiendaire, ...), nous avons choisi de ne concerver que l'ocr.

Ce notebook permet de créer un dataframe qui contiendra ces seules données (chapitre 2).

Dans un deuxième temps (chapitre 3), les données manquantes seront complétées par utilisation d'un outil d'océrisation sur les images IIT-CDIP (meilleure résolution que RVL-CDIP) correspondantes.


## 1. Préparation

In [1]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [34]:
import pandas as pd
from PIL import Image
import pytesseract
from tqdm import tqdm

## 2. Préparation de la base de données

In [45]:
df_ocr = pd.read_parquet(PATHS.processed_data / "df_iit_cdip_coll_xml_ltdlwocr_features.parquet")[["ocr_output"]]
df_ocr.isna().sum()

ocr_output    11356
dtype: int64

## 3. Complétion des ocr manquants

In [25]:
df_documents = pd.read_parquet(PATHS.metadata / "df_documents.parquet")
missing_ocrs = df_documents[df_documents.join(df_ocr).ocr_output.isna()]

In [30]:
image_paths = PATHS.data / pd.read_parquet(PATHS.metadata / "df_filepaths.parquet").iit_image_path

In [22]:
import pytesseract
try:
    version = pytesseract.get_tesseract_version()
    print("Tesseract version:", version)
    print("pytesseract est bien configuré.")
except Exception as e:
    print("Erreur:", e)

Tesseract version: 5.5.1
pytesseract est bien configuré.


In [23]:
# EN CAS D'ERREUR
# Si tesseract est bien installé mais ne se trouve pas dans le path, il est possible de définir son chemin avec cette commande:
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract" # Chemin à mettre à jour le cas echeant

try:
    version = pytesseract.get_tesseract_version()
    print("Tesseract version:", version)
    print("pytesseract est bien configuré.")
except Exception as e:
    print("Erreur:", e)

Tesseract version: 5.5.1
pytesseract est bien configuré.


In [24]:
def extract_text_from_image(image_path: str) -> str:
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

In [ ]:
# Environ 3h00 de calcul pour 13346+ lignes
ocrs = []
for file_path in tqdm(image_paths[missing_ocrs.index].values):
    ocrs.append(extract_text_from_image(file_path))
pd.DataFrame(ocrs, index=missing_ocrs.index)

 69%|██████████████████████████████████████████████████████████████████████████████████████████████▉                                           | 9182/13346 [2:08:58<29:28,  2.35it/s]

In [ ]:
recomputed_ocrs = pd.DataFrame(ocrs, index=missing_ocrs, columns = ["ocr_output"])

In [ ]:
df_ocr.combine_first(recomputed_ocrs).isna().sum()

### 3.2. Sauvegarde des DataFrames

In [ ]:
df_ocr.to_parquet(PATHS.processed_data / "df_raw_ocr.parquet")